 Testing output from an api run


In [1]:
from models.data_models import Entity, Labels
import rich, json, re
from tqdm import tqdm
from openai import AzureOpenAI
import azure.keyvault.secrets as azk
from azure.identity import DefaultAzureCredential
import os, asyncio
from openai_api import *

In [ ]:
def clean(text):
    text = text.strip().replace("\n", " ").replace("\r", "").replace("\t", " ")
    text = re.sub('\\s+', ' ', text)
    return text

In [ ]:
resumes = []

for file in tqdm(os.listdir(f'{os.getcwd()}/DATA/data_en')):
    with open(f'{os.getcwd()}/DATA/data_en/{file}','r') as f:
        contents = clean(f.read())
        resumes.append(contents)

with open('resumes.json','x') as j:
    j.write(json.dumps(resumes,indent=4,ensure_ascii=False))
        

In [2]:
with open('resumes.json','r') as f:
    resumes = json.loads(f.read())

In [ ]:
labeled_resumes = []

for i in tqdm(range(len(resumes))):
    text = resumes[i]
    api_labels = await send_request(text)

    labels = []
    for l in api_labels or []:
        label = l['type']
        content = l['text']
        start = text.find(content)
        if start == -1:
            continue
        end = start + len(content)
        labels.append(Label(start,end,label,content).to_dict())

    doc = Document(i,text,labels).to_dict()
    labeled_resumes.append(doc)

In [ ]:
with open('labeled_resumes.json','x') as j:
    j.write(json.dumps(labeled_resumes, indent=4,ensure_ascii=False))

In [ ]:
obj = await send_request(f'{os.getcwd()}/DATA/data_cs/1.txt')
contents = obj[0].replace("\n", " ").replace("\t", " ").replace("\r", " ") # type: ignore
labels = obj[1] # type: ignore
rich.print(obj)

In [ ]:
from models.label_models import Label, Document

In [ ]:
import json


def check_duplicates(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    label_count = 0
    dupl_count = 0
    for entry in data:

        if "labels" in entry and isinstance(entry["labels"], list):
            labels = entry["labels"]
            seen = set()
            duplicates = []
            
            for label in labels:
                label_count += 1
                label_tuple = tuple(label.items())  # Convert dict to tuple for hashable comparison
                if label_tuple in seen:
                    duplicates.append(label)
                else:
                    seen.add(label_tuple)
            
            if duplicates:
                dupl_count += len(duplicates)
                #for duplicate in duplicates: print(duplicate)
    print(dupl_count, label_count)


check_duplicates(f'{os.getcwd()}/labels_en.json')


In [ ]:
def format_entities(doc_contents:str,labels_list:list,doc_id:int=0):
    doc = doc_contents.replace("\n", " ").replace("\t", " ").replace("\r", " ")
    entities = labels_list
    labeled_ents = []
    for entity in entities:
        content = entity['text']
        label = entity['type']
        if content not in doc_contents:
            continue
        else:
            start = doc_contents.find(content)
            end = start + len(content)
            labeled_ent = Label(start,end,label,content).to_dict()
            labeled_ents.append(labeled_ent)
    labeled_doc = Document(doc_id,doc_contents, labeled_ents).to_dict()
    return labeled_doc




In [ ]:
jsonfile = json.dumps(new_data, indent=4)
with open('labels_en.json', 'w') as j:
    j.write(jsonfile)

In [ ]:
check_duplicates('labels_en.json')

# openai calls
```
async def main():
    labeled_docs = []
    for i in range(1,2456):
        label_obj = await send_request(f'{os.getcwd()}/data_src/{i}.txt')
        contents = label_obj[0] # type: ignore
        labels: list = label_obj[1]  # type: ignore
        formatted_labels = format_entities(contents,labels,i)
        labeled_docs.append(formatted_labels)
    with open(f'{os.getcwd()}/labels_en.json', 'w') as jsonfile:
        jsonfile.write(json.dumps(labeled_docs))
        

    for i in range(1,572):
        label_obj = await send_request(f'{os.getcwd()}/data_cs/{i}.txt')
        contents = label_obj[0] # type: ignore
        labels: list = label_obj[1]  # type: ignore
        formatted_labels = format_entities(contents,labels,i)
        labeled_docs.append(formatted_labels)
        if i % 10 == 0:
            with open(f'{os.getcwd()}/labels_to_{i}.json', 'w') as jsonfile:
                jsonfile.write(json.dumps(labeled_docs))
    with open(f'{os.getcwd()}/data_cs/labels_czech.json', 'w') as jsonfile:
        jsonfile.write(json.dumps(labeled_docs))
```

# from colab

In [ ]:
duplicate_count = 0
train = []
train_json = [] # TODO:learn how to write json hooks
for d in train_raw:
  debug_id,text,labels = d["id"],d["content"],d["labels"]
  unique_labels = []
  labels_json = []
  existing_labels = set()
  for l in labels:
    start, end, label, value = l["start"], l["end"], l["label"], l["value"]
    label_tuple = tuple(label)  # tuple conversion for hashable comparison
    if label_tuple in existing_labels:
      duplicate_count +=1
      continue;
    else:
      unique_labels.append(Label(start,end,label,value))
      labels_json.append(Label(start,end,label,value).to_dict()) # ! to_dict is not a built in 
  train.append(Document(debug_id, text, unique_labels))
  train_json.append(Document(debug_id, text, labels_json).to_dict())

print(duplicate_count)
with open('test_labels.json', 'w') as f:
    json.dump(train_json, f, indent=2)

In [ ]:
test = []
for d in test_raw:
  debug_id,text,labels = d["id"],d["content"],d["labels"]
  labels0 = []
  for l in labels:
    start, end, label, value = l["start"], l["end"], l["label"], l["value"]
    labels0.append(Label(start,end,label,value))
  test.append(Document(debug_id, text, labels0))